In [54]:
import pandas as pd
import statsmodels.api as sm
import datetime as datetime
import numpy as np

def zscore(df, col):
    col_mean = df[col].rolling(window=14, center=True).mean()
    col_std = df[col].rolling(window=14, center=True).std()
    return (df[col] - col_mean)/col_std

df = pd.read_csv('./data/11-26/daily_plutchik_threshold_5.csv', parse_dates=['month/day'])
approval_rating_df = pd.read_csv('./data/approval_polllist.csv', parse_dates=["enddate"])
topic_df = pd.read_excel('./data/TopicScoresPerDay_Final.xlsx')
sentiment_df = pd.read_csv('./data/12-6/daily_senti_corrected.csv', parse_dates=['month/day'])

In [55]:
df["Anticipation Z-Score"] = zscore(df, "Anticipation")
df["Trust Z-Score"] = zscore(df, "Trust")
df["Surprise Z-Score"] = zscore(df, "Surprise")
df["Sadness Z-Score"] = zscore(df, "Sadness")
df["Joy Z-Score"] = zscore(df, "Joy")
df["Fear Z-Score"] = zscore(df, "Fear")
df["Disgust Z-Score"] = zscore(df, "Disgust")
df["Anger Z-Score"] = zscore(df, "Anger")

start_date = datetime.datetime(2017, 7, 12, 0, 0)
end_date = datetime.datetime(2017, 12, 31, 0, 0)

dated_emotion_df = df[(df["month/day"] > start_date) & (df["month/day"] < end_date)]
original_emotions = dated_emotion_df[['Anger', 'Disgust', 'Fear', 'Joy', 'Sadness', 'Surprise', 'Trust', 'Anticipation']]
original_emotions.head()

,Anger,Disgust,Fear,Joy,Sadness,Surprise,Trust,Anticipation
66,0.025688,0.022018,0.036697,0.122936,0.027523,0.132110,0.620183,0.012844
67,0.015248,0.017789,0.054638,0.121982,0.034307,0.113088,0.635324,0.007624
68,0.013717,0.019204,0.046639,0.130316,0.038409,0.116598,0.626886,0.008230
69,0.022744,0.030814,0.038151,0.118855,0.031548,0.131328,0.620690,0.005869
70,0.022989,0.007663,0.034483,0.099617,0.034483,0.061303,0.735632,0.003831


In [56]:
dated_emotion_df = dated_emotion_df[['Anticipation Z-Score', 'Trust Z-Score', 'Surprise Z-Score', 
                             'Sadness Z-Score', 'Joy Z-Score', 'Disgust Z-Score', 'Anger Z-Score']]

truncated_approvals = approval_rating_df.loc[approval_rating_df['pollster'] == 'Ipsos']
truncated_approvals = truncated_approvals.loc[truncated_approvals['subgroup'] == 'All polls']
truncated_approvals = truncated_approvals[(truncated_approvals['enddate'] > start_date) & (truncated_approvals['enddate'] < end_date)]
truncated_approvals = truncated_approvals['adjusted_approve']

print(len(truncated_approvals), len(dated_emotion_df))
truncated_approvals = truncated_approvals.reset_index(drop=True)
dated_emotion_df = dated_emotion_df.reset_index(drop=True)
dated_emotion_df.head()

171 171


,Anticipation Z-Score,Trust Z-Score,Surprise Z-Score,Sadness Z-Score,Joy Z-Score,Disgust Z-Score,Anger Z-Score
0,2.252087,-0.827227,0.896344,-0.484259,0.977056,-0.307671,0.951454
1,0.667345,-0.652236,0.315559,0.635090,0.981600,-0.809876,-1.021932
2,0.859557,-0.902594,0.458054,1.209822,1.440038,-0.579125,-1.226268
3,0.061561,-1.197078,1.424133,0.206330,0.811716,1.217188,0.611709
4,-0.688659,1.712400,-1.867824,0.329168,-0.134768,-2.360562,0.725312


In [57]:
for (columnName, columnData) in sentiment_df.iteritems():
    if columnName != "month/day":
        sentiment_df[columnName+" Z-Score"] = zscore(sentiment_df, columnName)
sentiment_df = sentiment_df[(sentiment_df["month/day"] > start_date) & (sentiment_df["month/day"] < end_date)]
# sentiment_df = sentiment_df.reset_index(drop=True)
sentiment_df = sentiment_df[['pos_sentiment Z-Score', 'neg_sentiment Z-Score', 'neu_sentiment Z-Score']]
sentiment_df = sentiment_df.reset_index(drop=True)
sentiment_df.head()

,pos_sentiment Z-Score,neg_sentiment Z-Score,neu_sentiment Z-Score
0,1.339725,-0.882616,-0.535477
1,0.723188,-0.960567,0.695292
2,1.445375,-1.252709,0.201804
3,-0.453674,0.663924,-0.590644
4,0.058015,0.762438,-1.515081


In [58]:
emotions_only = sm.OLS(truncated_approvals, dated_emotion_df).fit()
emotions_only.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:       adjusted_approve   R-squared (uncentered):                   0.002
Model:                            OLS   Adj. R-squared (uncentered):             -0.041
Method:                 Least Squares   F-statistic:                            0.04647
Date:                Mon, 09 Dec 2019   Prob (F-statistic):                        1.00
Time:                        10:47:58   Log-Likelihood:                         -862.79
No. Observations:                 171   AIC:                                      1740.
Df Residuals:                     164   BIC:                                      1762.
Df Model:                           7                                                  
Covariance Type:            nonrobust                                                  
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Anticipation Z-Score    -1.1108      3.294     -0.337      0.736      -7.615       5.394
Trust Z-Score           -0.7977      4.976     -0.160      0.873     -10.623       9.028
Surprise Z-Score        -0.6491      4.022     -0.161      0.872      -8.591       7.293
Sadness Z-Score         -1.3874      3.473     -0.400      0.690      -8.244       5.470
Joy Z-Score              0.6266      4.344      0.144      0.885      -7.952       9.205
Disgust Z-Score         -0.1856      3.768     -0.049      0.961      -7.626       7.255
Anger Z-Score           -0.4234      3.597     -0.118      0.906      -7.526       6.679
==============================================================================
Omnibus:                        0.398   Durbin-Watson:                   0.005
Prob(Omnibus):                  0.819   Jarque-Bera (JB):                0.218
Skew:                           0.080   Prob(JB):                        0.897
Kurtosis:                       3.071   Cond. No.                         3.27
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [59]:
sentiment_only = sm.OLS(truncated_approvals, sentiment_df).fit()
sentiment_only.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:       adjusted_approve   R-squared (uncentered):                   0.002
Model:                            OLS   Adj. R-squared (uncentered):             -0.016
Method:                 Least Squares   F-statistic:                             0.1037
Date:                Mon, 09 Dec 2019   Prob (F-statistic):                       0.958
Time:                        10:47:59   Log-Likelihood:                         -862.80
No. Observations:                 171   AIC:                                      1732.
Df Residuals:                     168   BIC:                                      1741.
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
pos_sentiment Z-Score    -7.4234     13.883     -0.535      0.594     -34.830      19.983
neg_sentiment Z-Score    -7.6727     14.627     -0.525      0.601     -36.550      21.205
neu_sentiment Z-Score    -4.2740      7.796     -0.548      0.584     -19.665      11.117
==============================================================================
Omnibus:                        1.296   Durbin-Watson:                   0.004
Prob(Omnibus):                  0.523   Jarque-Bera (JB):                0.993
Skew:                           0.175   Prob(JB):                        0.609
Kurtosis:                       3.132   Cond. No.                         9.37
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [60]:
sentiment_and_emotion = pd.concat([sentiment_df, dated_emotion_df], axis=1)
sentiment_and_emotion_only = sm.OLS(truncated_approvals, sentiment_and_emotion).fit()
sentiment_and_emotion_only.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:       adjusted_approve   R-squared (uncentered):                   0.004
Model:                            OLS   Adj. R-squared (uncentered):             -0.058
Method:                 Least Squares   F-statistic:                            0.06235
Date:                Mon, 09 Dec 2019   Prob (F-statistic):                        1.00
Time:                        10:48:00   Log-Likelihood:                         -862.63
No. Observations:                 171   AIC:                                      1745.
Df Residuals:                     161   BIC:                                      1777.
Df Model:                          10                                                  
Covariance Type:            nonrobust                                                  
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
pos_sentiment Z-Score    -8.0352     15.021     -0.535      0.593     -37.699      21.629
neg_sentiment Z-Score    -7.9574     15.735     -0.506      0.614     -39.030      23.115
neu_sentiment Z-Score    -4.5405      8.523     -0.533      0.595     -21.372      12.291
Anticipation Z-Score     -0.9818      3.395     -0.289      0.773      -7.686       5.723
Trust Z-Score             0.1559      5.418      0.029      0.977     -10.543      10.855
Surprise Z-Score         -0.1040      4.479     -0.023      0.982      -8.949       8.741
Sadness Z-Score          -1.0881      3.551     -0.306      0.760      -8.100       5.924
Joy Z-Score               1.1851      4.645      0.255      0.799      -7.988      10.359
Disgust Z-Score           0.1188      3.970      0.030      0.976      -7.721       7.959
Anger Z-Score            -0.5441      3.660     -0.149      0.882      -7.771       6.683
==============================================================================
Omnibus:                        1.261   Durbin-Watson:                   0.009
Prob(Omnibus):                  0.532   Jarque-Bera (JB):                0.884
Skew:                          -0.073   Prob(JB):                        0.643
Kurtosis:                       3.321   Cond. No.                         12.9
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [61]:
topic_df = topic_df.drop(['Filename', 'Segment', 'WC', 'WPS', 'Sixltr', 'Dic', 
                          'AllPunc', 'Period', 'Comma', 'Colon', 'SemiC', 'QMark', 
                          'Exclam', 'Dash', 'Quote', 'Apostro', 'Parenth', 'OtherP', 
                          'Em1', 'Em2', 'Em3', 'Em4', 'Em5', 'Em6', 'Em7', 'Em8', 'Gallup Value'], axis=1)

for (columnName, columnData) in topic_df.iteritems():
    topic_df[columnName + " Z-Score"] = zscore(topic_df, columnName)
    
topic_df = topic_df[66:237]
topic_df.fillna(0, inplace=True)
topic_df.head()

,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9,topic10,...,topic492 Z-Score,topic493 Z-Score,topic494 Z-Score,topic495 Z-Score,topic496 Z-Score,topic497 Z-Score,topic498 Z-Score,topic499 Z-Score,topic500 Z-Score,topic501 Z-Score
66,6.46,0.0,0.00,0.05,0.0,0.01,0.64,0.57,0.19,0.65,...,0.194023,2.163111,1.089956,-0.491032,1.429682,0.413594,-0.702665,1.065521,1.432840,-0.507075
67,6.12,0.0,0.00,0.04,0.0,0.01,0.46,0.65,0.08,0.49,...,0.769534,-0.151553,-0.433117,-0.479883,0.629919,1.120638,-0.401764,-1.296174,-1.321625,-0.372797
68,6.20,0.0,0.01,0.01,0.0,0.02,0.61,0.51,0.10,0.69,...,0.461597,-1.572928,-0.890207,1.424433,1.097163,1.309574,-0.475466,-1.355222,-2.163202,-0.066748
69,6.08,0.0,0.00,0.03,0.0,0.02,0.47,0.50,0.07,0.67,...,-0.503236,0.630641,2.372378,2.064031,-0.495156,-0.901418,-0.321563,0.105594,0.360875,-0.514077
70,5.97,0.0,0.00,0.06,0.0,0.02,0.45,0.63,0.06,0.21,...,-0.493518,-0.165357,-1.091842,-0.820724,-0.338300,-1.210769,-1.583182,1.308417,0.067327,-0.723097


In [62]:
filter_col = [col for col in topic_df if col.endswith("Z-Score")]
topic_df = topic_df[filter_col]
topic_df.head()

,topic1 Z-Score,topic2 Z-Score,topic3 Z-Score,topic4 Z-Score,topic5 Z-Score,topic6 Z-Score,topic7 Z-Score,topic8 Z-Score,topic9 Z-Score,topic10 Z-Score,...,topic492 Z-Score,topic493 Z-Score,topic494 Z-Score,topic495 Z-Score,topic496 Z-Score,topic497 Z-Score,topic498 Z-Score,topic499 Z-Score,topic500 Z-Score,topic501 Z-Score
66,1.021734,0.0,-0.267261,1.519979e+00,-0.267261,-0.424212,0.858944,0.497919,2.562713,0.389045,...,0.194023,2.163111,1.089956,-0.491032,1.429682,0.413594,-0.702665,1.065521,1.432840,-0.507075
67,-0.084203,0.0,-0.267261,7.318418e-01,-0.393398,-0.424212,-0.699777,1.011870,-0.705266,-0.829156,...,0.769534,-0.151553,-0.433117,-0.479883,0.629919,1.120638,-0.401764,-1.296174,-1.321625,-0.372797
68,0.190444,0.0,3.474396,-1.537412e+00,-0.393398,0.755929,0.610476,0.015365,-0.174485,0.724042,...,0.461597,-1.572928,-0.890207,1.424433,1.097163,1.309574,-0.475466,-1.355222,-2.163202,-0.066748
69,-0.132464,0.0,-0.267261,-2.133588e-15,-0.393398,0.862958,-0.431890,-0.089888,-1.128386,0.471500,...,-0.503236,0.630641,2.372378,2.064031,-0.495156,-0.901418,-0.321563,0.105594,0.360875,-0.514077
70,-0.498835,0.0,-0.267261,2.308116e+00,-0.393398,0.862958,-0.492359,0.877262,-1.508512,-2.808116,...,-0.493518,-0.165357,-1.091842,-0.820724,-0.338300,-1.210769,-1.583182,1.308417,0.067327,-0.723097


In [70]:
import csv

significant_topics = list()
with open('./significant_topics.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        print(row)
        significant_topics.append(row[0])

significant_topics_df = topic_df[significant_topics]
significant_topics_df.head()

['topic448 Z-Score']
['topic492 Z-Score']
['topic337 Z-Score']
['topic374 Z-Score']
['topic473 Z-Score']
['topic478 Z-Score']
['topic104 Z-Score']
['topic10 Z-Score']
['topic289 Z-Score']
['topic386 Z-Score']
['topic142 Z-Score']
['topic98 Z-Score']
['topic399 Z-Score']
['topic461 Z-Score']
['topic162 Z-Score']
['topic231 Z-Score']
['topic203 Z-Score']
['topic157 Z-Score']
['topic380 Z-Score']
['topic404 Z-Score']
['topic452 Z-Score']
['topic382 Z-Score']
['topic333 Z-Score']
['topic346 Z-Score']
['topic319 Z-Score']
['topic148 Z-Score']
['topic40 Z-Score']
['topic79 Z-Score']
['topic491 Z-Score']
['topic115 Z-Score']
['topic262 Z-Score']
['topic112 Z-Score']
['topic377 Z-Score']
['topic366 Z-Score']
['topic218 Z-Score']
['topic369 Z-Score']
['topic70 Z-Score']
['topic237 Z-Score']
['topic117 Z-Score']
['topic105 Z-Score']
['topic6 Z-Score']
['topic265 Z-Score']
['topic321 Z-Score']
['topic126 Z-Score']
['topic39 Z-Score']
['topic211 Z-Score']
['topic482 Z-Score']
['topic221 Z-Score']


,topic448 Z-Score,topic492 Z-Score,topic337 Z-Score,topic374 Z-Score,topic473 Z-Score,topic478 Z-Score,topic104 Z-Score,topic10 Z-Score,topic289 Z-Score,topic386 Z-Score,...,topic209 Z-Score,topic352 Z-Score,topic476 Z-Score,topic469 Z-Score,topic164 Z-Score,topic167 Z-Score,topic358 Z-Score,topic174 Z-Score,topic67 Z-Score,topic437 Z-Score
0,-0.910524,0.194023,0.262874,-0.735523,-1.287484,0.498039,1.898131,0.389045,0.951558,1.143782,...,0.250292,-0.601396,0.821961,0.418879,-0.728331,0.705266,0.000000,-0.160924,-8.451005e-01,-0.335111
1,0.123091,0.769534,-0.144453,-0.249152,-0.694632,-0.216239,-0.762905,-0.829156,-0.873348,0.519142,...,-0.937465,1.830916,-0.606387,1.131229,0.826013,-0.399190,0.058428,-0.655906,3.675118e-01,0.229882
2,-0.081582,0.461597,0.464226,-0.688115,-0.662536,-0.016178,-1.935651,0.724042,-0.361046,-0.020333,...,-0.692823,0.329683,-1.545618,-0.226718,1.263692,-0.112415,0.150493,-0.844817,2.049498e+00,1.274386
3,-0.174115,-0.503236,1.845200,0.394577,-0.306614,-0.111482,0.779406,0.471500,0.630041,0.464364,...,1.405537,-0.984511,-0.468532,0.216204,-0.824463,0.318689,-0.250906,-0.149085,2.190326e-15,-0.382905
4,2.810797,-0.493518,-2.148618,2.773039,1.763879,-2.599943,1.202340,-2.808116,2.002129,1.608392,...,2.469377,1.407731,2.256563,-2.299332,1.585361,-1.274755,0.031961,2.981707,1.325698e+00,-1.749701


In [71]:
topic_df = topic_df.reset_index(drop=True)
topics_only = sm.OLS(truncated_approvals, significant_topics_df).fit()
topics_only.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:       adjusted_approve   R-squared (uncentered):                   0.104
Model:                            OLS   Adj. R-squared (uncentered):             -2.261
Method:                 Least Squares   F-statistic:                            0.04383
Date:                Mon, 09 Dec 2019   Prob (F-statistic):                        1.00
Time:                        11:25:42   Log-Likelihood:                         -853.61
No. Observations:                 171   AIC:                                      1955.
Df Residuals:                      47   BIC:                                      2345.
Df Model:                         124                                                  
Covariance Type:            nonrobust                                                  
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
topic448 Z-Score     6.2964     11.307      0.557      0.580     -16.450      29.042
topic492 Z-Score     1.5836     11.006      0.144      0.886     -20.558      23.725
topic337 Z-Score    -4.2580     20.236     -0.210      0.834     -44.967      36.451
topic374 Z-Score    -0.3120     12.250     -0.025      0.980     -24.955      24.331
topic473 Z-Score     6.8987     15.338      0.450      0.655     -23.957      37.755
topic478 Z-Score     8.0572     21.500      0.375      0.710     -35.196      51.310
topic104 Z-Score    -1.8690     20.271     -0.092      0.927     -42.648      38.910
topic10 Z-Score     -6.8805     20.296     -0.339      0.736     -47.710      33.949
topic289 Z-Score     2.8363     14.938      0.190      0.850     -27.215      32.888
topic386 Z-Score    -9.5645     22.772     -0.420      0.676     -55.377      36.248
topic142 Z-Score     0.4069     15.312      0.027      0.979     -30.396      31.210
topic98 Z-Score      5.0736     16.262      0.312      0.756     -27.641      37.789
topic399 Z-Score     3.5862     28.953      0.124      0.902     -54.659      61.831
topic461 Z-Score     8.5068     14.096      0.603      0.549     -19.851      36.865
topic162 Z-Score    -5.0451     13.875     -0.364      0.718     -32.959      22.869
topic231 Z-Score    -0.4710     11.362     -0.041      0.967     -23.329      22.387
topic203 Z-Score    -2.8318     12.064     -0.235      0.815     -27.101      21.437
topic157 Z-Score     4.6207     13.512      0.342      0.734     -22.563      31.804
topic380 Z-Score    -8.0618     14.533     -0.555      0.582     -37.298      21.174
topic404 Z-Score   -11.5251     21.879     -0.527      0.601     -55.541      32.490
topic452 Z-Score    -3.2432     15.363     -0.211      0.834     -34.150      27.664
topic382 Z-Score     2.4665     35.275      0.070      0.945     -68.498      73.431
topic333 Z-Score    -0.0368     18.138     -0.002      0.998     -36.526      36.452
topic346 Z-Score    -1.6163      9.710     -0.166      0.869     -21.151      17.918
topic319 Z-Score     8.0411     15.687      0.513      0.611     -23.517      39.600
topic148 Z-Score     4.6926     13.492      0.348      0.730     -22.450      31.836
topic40 Z-Score     -1.9787     15.510     -0.128      0.899     -33.181      29.224
topic79 Z-Score      0.7128     20.101      0.035      0.972     -39.725      41.150
topic491 Z-Score    -4.0002     20.186     -0.198      0.844     -44.608      36.608
topic115 Z-Score     7.3030     40.934      0.178      0.859     -75.045      89.651
topic262 Z-Score     3.2205     34.996      0.092      0.927     -67.183      73.624
topic112 Z-Score    -3.3482     14.104     -0.237      0.813     -31.722      25.025
topic377 

In [72]:
topics_and_sentiment = pd.concat([sentiment_df, significant_topics_df], axis=1)
topics_and_sentiment_only = sm.OLS(truncated_approvals, topics_and_sentiment).fit()
topics_and_sentiment_only.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:       adjusted_approve   R-squared (uncentered):                   0.118
Model:                            OLS   Adj. R-squared (uncentered):             -2.428
Method:                 Least Squares   F-statistic:                            0.04631
Date:                Mon, 09 Dec 2019   Prob (F-statistic):                        1.00
Time:                        11:27:01   Log-Likelihood:                         -852.24
No. Observations:                 171   AIC:                                      1958.
Df Residuals:                      44   BIC:                                      2357.
Df Model:                         127                                                  
Covariance Type:            nonrobust                                                  
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
pos_sentiment Z-Score    -9.9389     57.712     -0.172      0.864    -126.250     106.372
neg_sentiment Z-Score   -21.1948     58.088     -0.365      0.717    -138.263      95.873
neu_sentiment Z-Score   -18.0591     30.554     -0.591      0.558     -79.636      43.518
topic448 Z-Score          6.0786     11.757      0.517      0.608     -17.617      29.774
topic492 Z-Score          1.6047     11.487      0.140      0.890     -21.546      24.756
topic337 Z-Score         -2.7356     21.053     -0.130      0.897     -45.165      39.694
topic374 Z-Score         -3.0290     14.480     -0.209      0.835     -32.212      26.154
topic473 Z-Score          9.9590     17.475      0.570      0.572     -25.260      45.178
topic478 Z-Score         10.3717     23.552      0.440      0.662     -37.094      57.837
topic104 Z-Score          0.8068     22.928      0.035      0.972     -45.401      47.015
topic10 Z-Score         -10.3229     21.571     -0.479      0.635     -53.797      33.151
topic289 Z-Score          0.5730     15.777      0.036      0.971     -31.224      32.370
topic386 Z-Score         -4.8554     24.483     -0.198      0.844     -54.198      44.487
topic142 Z-Score          0.8816     16.057      0.055      0.956     -31.479      33.242
topic98 Z-Score           4.9023     16.949      0.289      0.774     -29.255      39.060
topic399 Z-Score         -0.7885     30.891     -0.026      0.980     -63.046      61.469
topic461 Z-Score          9.0095     14.993      0.601      0.551     -21.207      39.226
topic162 Z-Score         -6.0295     15.021     -0.401      0.690     -36.302      24.243
topic231 Z-Score          1.7048     12.349      0.138      0.891     -23.184      26.593
topic203 Z-Score         -5.4876     13.218     -0.415      0.680     -32.126      21.151
topic157 Z-Score          4.0265     14.561      0.277      0.783     -25.320      33.373
topic380 Z-Score         -9.6206     15.676     -0.614      0.543     -41.214      21.973
topic404 Z-Score        -11.5055     24.585     -0.468      0.642     -61.053      38.042
topic452 Z-Score         -6.3616     16.471     -0.386      0.701     -39.557      26.834
topic382 Z-Score         -1.6447     36.807     -0.045      0.965     -75.824      72.534
topic333 Z-Score         -0.1351     19.456     -0.007      0.994     -39.346      39.076
topic346 Z-Score          0.3910     10.366      0.038      0.970     -20.501      21.282
topic319 Z-Score         10.9570     16.644      0.658      0.514     -22.587      44.502
topic148 Z-Score          8.3458     14.614      0.571      0.571     -21.106      37.798
topic40 Z-Score          -0.5906     16.189     -0.036      0.971     -33.218      32.037
topic79 Z-Scor

In [73]:
topics_and_emotion = pd.concat([dated_emotion_df, significant_topics_df], axis=1)
topics_and_emotions_only = sm.OLS(truncated_approvals, topics_and_emotion).fit()
topics_and_emotions_only.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:       adjusted_approve   R-squared (uncentered):                   0.134
Model:                            OLS   Adj. R-squared (uncentered):             -2.704
Method:                 Least Squares   F-statistic:                            0.04706
Date:                Mon, 09 Dec 2019   Prob (F-statistic):                        1.00
Time:                        11:28:49   Log-Likelihood:                         -850.71
No. Observations:                 171   AIC:                                      1963.
Df Residuals:                      40   BIC:                                      2375.
Df Model:                         131                                                  
Covariance Type:            nonrobust                                                  
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Anticipation Z-Score    -1.7204     13.492     -0.128      0.899     -28.988      25.547
Trust Z-Score          -19.3367     25.018     -0.773      0.444     -69.900      31.227
Surprise Z-Score       -10.9450     19.129     -0.572      0.570     -49.606      27.716
Sadness Z-Score        -11.5401     14.323     -0.806      0.425     -40.488      17.408
Joy Z-Score              4.7192     18.853      0.250      0.804     -33.384      42.822
Disgust Z-Score         -8.0014     15.974     -0.501      0.619     -40.286      24.283
Anger Z-Score           -5.5322     15.981     -0.346      0.731     -37.830      26.766
topic448 Z-Score         0.6638     13.817      0.048      0.962     -27.262      28.590
topic492 Z-Score         1.3715     11.970      0.115      0.909     -22.820      25.563
topic337 Z-Score        -7.8644     23.209     -0.339      0.736     -54.772      39.043
topic374 Z-Score        -1.9988     14.137     -0.141      0.888     -30.571      26.574
topic473 Z-Score         5.1925     18.097      0.287      0.776     -31.384      41.769
topic478 Z-Score         7.9172     23.376      0.339      0.737     -39.327      55.162
topic104 Z-Score       -12.6152     26.791     -0.471      0.640     -66.761      41.530
topic10 Z-Score         -4.2434     22.781     -0.186      0.853     -50.285      41.798
topic289 Z-Score         1.6724     16.320      0.102      0.919     -31.312      34.657
topic386 Z-Score        -5.1900     24.840     -0.209      0.836     -55.393      45.013
topic142 Z-Score        -2.3069     17.341     -0.133      0.895     -37.355      32.741
topic98 Z-Score          8.3258     18.435      0.452      0.654     -28.933      45.584
topic399 Z-Score         2.0204     33.827      0.060      0.953     -66.347      70.388
topic461 Z-Score        10.1988     15.607      0.653      0.517     -21.344      41.741
topic162 Z-Score        -3.4997     14.908     -0.235      0.816     -33.629      26.630
topic231 Z-Score        -2.6381     13.041     -0.202      0.841     -28.995      23.719
topic203 Z-Score        -2.9863     13.572     -0.220      0.827     -30.416      24.443
topic157 Z-Score         6.1418     15.392      0.399      0.692     -24.966      37.250
topic380 Z-Score        -8.8186     15.969     -0.552      0.584     -41.094      23.457
topic404 Z-Score        -8.6583     25.705     -0.337      0.738     -60.611      43.294
topic452 Z-Score        -1.8990     17.934     -0.106      0.916     -38.144      34.346
topic382 Z-Score        -5.8054     39.799     -0.146      0.885     -86.242      74.631
topic333 Z-Score        -6.4597     21.470     -0.301      0.765     -49.853      36.934
topic346 Z-Score        -3.2446     11.049     

In [74]:
topics_sentiment_and_emotion = pd.concat([sentiment_df, dated_emotion_df, significant_topics_df], axis=1)
full_model = sm.OLS(truncated_approvals, topics_sentiment_and_emotion).fit()
full_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:       adjusted_approve   R-squared (uncentered):                   0.150
Model:                            OLS   Adj. R-squared (uncentered):             -2.927
Method:                 Least Squares   F-statistic:                            0.04886
Date:                Mon, 09 Dec 2019   Prob (F-statistic):                        1.00
Time:                        11:29:44   Log-Likelihood:                         -849.03
No. Observations:                 171   AIC:                                      1966.
Df Residuals:                      37   BIC:                                      2387.
Df Model:                         134                                                  
Covariance Type:            nonrobust                                                  
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
pos_sentiment Z-Score   -11.0276     67.544     -0.163      0.871    -147.885     125.829
neg_sentiment Z-Score   -28.8019     71.081     -0.405      0.688    -172.826     115.222
neu_sentiment Z-Score   -19.9692     36.973     -0.540      0.592     -94.883      54.945
Anticipation Z-Score     -2.0595     14.199     -0.145      0.885     -30.830      26.711
Trust Z-Score           -16.2035     27.410     -0.591      0.558     -71.741      39.334
Surprise Z-Score        -11.6218     21.505     -0.540      0.592     -55.196      31.952
Sadness Z-Score         -10.4988     15.368     -0.683      0.499     -41.637      20.640
Joy Z-Score               9.6170     21.049      0.457      0.650     -33.032      52.266
Disgust Z-Score          -2.4876     17.859     -0.139      0.890     -38.673      33.698
Anger Z-Score            -3.4148     16.849     -0.203      0.841     -37.554      30.725
topic448 Z-Score         -0.5760     14.399     -0.040      0.968     -29.751      28.599
topic492 Z-Score          2.4166     12.650      0.191      0.850     -23.215      28.048
topic337 Z-Score         -5.8118     24.238     -0.240      0.812     -54.923      43.299
topic374 Z-Score         -4.4409     16.617     -0.267      0.791     -38.110      29.228
topic473 Z-Score          9.2649     20.337      0.456      0.651     -31.942      50.472
topic478 Z-Score         11.3354     25.922      0.437      0.664     -41.188      63.859
topic104 Z-Score        -11.3082     29.073     -0.389      0.700     -70.216      47.599
topic10 Z-Score          -9.9565     24.962     -0.399      0.692     -60.535      40.622
topic289 Z-Score          0.3868     17.492      0.022      0.982     -35.055      35.829
topic386 Z-Score          0.8940     27.068      0.033      0.974     -53.951      55.739
topic142 Z-Score          0.3801     18.258      0.021      0.984     -36.614      37.374
topic98 Z-Score           7.3374     19.764      0.371      0.713     -32.709      47.384
topic399 Z-Score         -1.8040     36.762     -0.049      0.961     -76.291      72.683
topic461 Z-Score          8.7639     16.719      0.524      0.603     -25.112      42.640
topic162 Z-Score         -3.3054     16.444     -0.201      0.842     -36.624      30.013
topic231 Z-Score          0.4148     14.159      0.029      0.977     -28.274      29.104
topic203 Z-Score         -7.0087     15.436     -0.454      0.652     -38.285      24.268
topic157 Z-Score          7.6240     17.294      0.441      0.662     -27.418      42.666
topic380 Z-Score        -10.0497     17.082     -0.588      0.560     -44.661      24.561
topic404 Z-Score         -6.2115     27.976     -0.222      0.826     -62.896      50.473
topic452 Z-Sco

In [224]:
unnormalized_df = dated_emotion_df[['Anticipation', 'Trust', 'Sadness', 'Joy', 'Fear', 'Disgust', 'Anger']]
unnormalized_extra_df = dated_emotion_df[['Anticipation', 'Trust', 'Surprise', 'Sadness', 'Joy', 'Fear', 'Disgust', 'Anger']]
dated_emotion_df = dated_emotion_df[['Anticipation Z-Score', 'Trust Z-Score', 'Surprise Z-Score', 
                             'Sadness Z-Score', 'Joy Z-Score', 'Disgust Z-Score', 'Anger Z-Score']]
dated_emotion_df.head()

,Anticipation Z-Score,Trust Z-Score,Surprise Z-Score,Sadness Z-Score,Joy Z-Score,Disgust Z-Score,Anger Z-Score
66,2.252087,-0.827227,0.896344,-0.484259,0.977056,-0.307671,0.951454
67,0.667345,-0.652236,0.315559,0.635090,0.981600,-0.809876,-1.021932
68,0.859557,-0.902594,0.458054,1.209822,1.440038,-0.579125,-1.226268
69,0.061561,-1.197078,1.424133,0.206330,0.811716,1.217188,0.611709
70,-0.688659,1.712400,-1.867824,0.329168,-0.134768,-2.360562,0.725312
